In [2]:
from paceutils import Utilization
import pandas as pd

In [3]:
u = Utilization()

In [4]:
def build_csv_date_spread(func, table, facility_col, start, end):
    if table == "authorizations":
        where_filter = "WHERE service_type IS 'Adult Day Center Attendance'"
    elif table == "inpatient":
        where_filter = "WHERE admission_type = 'Acute Hospital' OR admission_type = 'Psych Unit / Facility'"
    else:
        where_filter = ""
    
    df = u.dataframe_query(f"""SELECT DISTINCT({facility_col}) FROM {table} {where_filter};""")
    
    for date in pd.date_range(start, end, freq='M'):
        start_date = pd.to_datetime(f"{date.year}-{date.month}-01").date()
        params = (str(start_date), str(date.date())) 
        
        df = df.merge(func(params), on=facility_col, how="left").fillna(0)
    
    df.sort_values(by=facility_col, inplace=True)

    return df

In [5]:
def build_pressure_wound_csv(facility_type, start, end):
    df = u.dataframe_query("""SELECT DISTINCT(living_detail) FROM wounds
        WHERE living_situation IS ?""", [facility_type])
    df["living_detail"].fillna("Unknown", inplace=True)
    for date in pd.date_range(start, end, freq='M'):
        start_date = pd.to_datetime(f"{date.year}-{date.month}-01").date()
        params = (str(start_date), str(date.date())) 
        
        df = df.merge(pressure_ulcers_at_facility(facility_type, params), on="living_detail", how="left").fillna(0)
        
    df.sort_values(by="living_detail", inplace=True)
    return df

## Vendor Information

### Alfs
+ Census on 1st of Month **(done)**
+ ER/Hosp Admits **(done)**

### Day Centers
+ Census on 1st of Month **(done)**

### Home Care
+ Ppts Services or Hours Provided
+ Cancelations

### Hospital
+ Admissions **(done)**
+ Readmissions **(done)**
+ Resulting in Readmissions **(done)**
+ Hospital-Acquired Infections **(done)**

### Nursing Facilties
+ Census on 1st of Month **(done)**
+ ER/Hosp Admits **(done)**
+ Pressure Ulcers

## Alfs

**Census on 1st of Month**

In [6]:
def alf_census_on_date(params):
    return u.dataframe_query("""SELECT * FROM alfs
                            WHERE (discharge_date >= ?
                            OR discharge_date IS NULL)
                            AND admission_date <= ?;""",
                            params).groupby("facility_name").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                           f"census-{params[0][:7]}"})

In [7]:
alf_census_on_date(("2019-01-01", "2019-01-30"))

,facility_name,census-2019-01
0,Arbor Hill Assisted Living Community,5
1,Autumn Villa,1
2,Darlington Memory Lane,2
3,Highlands on the East Side,7
4,Spring Villa Dementia Care,5
5,St. Elizabeth Court,7
6,St. Germain Assisted Living,1
7,The Villa at Saint Antoine,1


**ER/Hosp Admits**

In [8]:
def alf_to_hosp(month_params):
    return u.dataframe_query("""SELECT * FROM alfs
            WHERE discharge_date BETWEEN ? AND ?
            AND discharge_type='Hospital/ER';""",
            params=month_params).groupby("facility_name").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                       f"hosp_admits-{month_params[0]}"})

In [9]:
alf_to_hosp(("2019-01-01", "2019-01-31"))

,facility_name,hosp_admits-2019-01-01
0,St. Elizabeth Court,1


### Nursing Homes

**Census on 1st of Month**

In [12]:
def nf_census_on_date(params):
    return u.dataframe_query("""SELECT * FROM inpatient
                            WHERE (discharge_date >= ?
                            OR discharge_date IS NULL)
                            AND admission_date <= ?;""",
                            params).groupby("facility").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                           f"census-{params[0][:7]}"})

In [13]:
nf_census_on_date(("2018-11-01", "2018-11-30"))

,facility,census-2018-11
0,Berkshire Place Nursing and Rehab,9
1,Crestwood Nursing & Rehabilitation Center,2
2,Firsthealth of the Carolinas,1
3,Hallworth House,5
4,Landmark Medical Center,2
5,Linn Health Care Center,5
6,Our Lady Of Fatima Hospital,1
7,Rhode Island Hospital,4
8,Riverview Healthcare,2
9,Roger Williams Medical Center,3


**ER/Hosp Admits**

In [11]:
def nf_to_hosp(month_params):
    return u.dataframe_query("""SELECT * FROM nursing_home
            WHERE discharge_date BETWEEN ? AND ?
            AND discharge_disposition IS 'Acute care hospital or psychiatric facility';""",
            params=month_params).groupby("facility").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                       f"hosp_admits-{month_params[0]}"})

### Hospital Admissions

**Admissions**

In [12]:
def hosp_admissions(params):
    return u.dataframe_query(f"""SELECT * FROM inpatient
                    WHERE admission_date BETWEEN ? AND ?;""",
                    params).groupby("facility").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                       f"admissions-{params[0][:7]}"})

In [13]:
hosp_admissions(("2019-01-01", "2019-01-31"))

,facility,admissions-2019-01
0,Bayberry Commons Nursing & Rehabilitation Center,2
1,Berkshire Place Nursing and Rehab,10
2,Hallworth House,3
3,Kent County Memorial Hospital,4
4,Landmark Medical Center,4
5,Rhode Island Hospital,10
6,Roger Williams Medical Center,6
7,Saint Antoine Residence,1
8,Scandinavian Home,2
9,St. Elizabeth Manor,1


**Hospital Resulting in 30 Day Readmissions**

In [14]:
def resulting_30_day_hosp_count(params):
    return u.admissions_resulting_in_30day_df(params, "inpatient").groupby("facility").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                       f"results_in_30dr-{params[0][:7]}"})

In [15]:
resulting_30_day_hosp_count(("2019-01-01", "2019-01-31"))

,facility,results_in_30dr-2019-01
0,Berkshire Place Nursing and Rehab,4
1,Hallworth House,2
2,Landmark Medical Center,2
3,Rhode Island Hospital,4
4,Roger Williams Medical Center,2
5,Scandinavian Home,1
6,Steere House Nursing & Rehabilitation,2
7,The Miriam Hospital,1
8,West Shore Health Center,1


**Hospital 30 Day Readmissions**

In [16]:
def readmit_30_day_hosp_count(params):
    return u.admissions_30day_readmit_df(params, "inpatient").groupby("facility").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                       f"30dr-{params[0][:7]}"})

In [17]:
readmit_30_day_hosp_count(("2019-01-01", "2019-01-31"))

,facility,30dr-2019-01
0,Berkshire Place Nursing and Rehab,4
1,Hallworth House,1
2,Landmark Medical Center,2
3,Rhode Island Hospital,4
4,Roger Williams Medical Center,3
5,Scandinavian Home,1
6,Steere House Nursing & Rehabilitation,2
7,West Shore Health Center,2


**Hospital Aquired Infections**

In [18]:
def infections_by_hosp(params):
    infections = u.dataframe_query("""SELECT * FROM infections
                        WHERE where_infection_was_acquired = 'Hospital'
                        AND date_time_occurred BETWEEN ? AND ?;""",
                                     params)
    infections_with_hosp = f"""
                    SELECT infections.member_id, admission_date, discharge_date, facility, los, date_time_occurred
                    FROM infections
                    LEFT JOIN inpatient ut on infections.member_id=ut.member_id
                    WHERE where_infection_was_acquired = 'Hospital'
                    AND date_time_occurred BETWEEN ? AND ?;"""
    
    df = u.dataframe_query(infections_with_hosp, params)

    df["discharge_date"] = pd.to_datetime(df["discharge_date"]).dt.date
    df["date_time_occurred"] = pd.to_datetime(df["date_time_occurred"]).dt.date
    
    df["days_between_hosp_and_inf"] = abs(df["date_time_occurred"] - df["discharge_date"])
    
    try:
        hosp_within_one_weeks = df[(df["days_between_hosp_and_inf"].dt.days <= 7)].copy()
        
    except AttributeError:
        return pd.DataFrame.from_dict({"facility":["None"], "infections":["None"]})
    
    hosp_within_one_weeks.sort_values("days_between_hosp_and_inf", inplace=True)
    hosp_within_one_weeks.drop_duplicates(["member_id", "date_time_occurred"], keep="first", inplace=True)
    infections[-infections["member_id"].isin(hosp_within_one_weeks["member_id"])].to_csv("hospital_inf_without_visit.csv", index=False)
    return hosp_within_one_weeks.groupby("facility").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                             "infections"})

In [19]:
infections_by_hosp(("2017-10-01", "2019-10-31"))

,facility,infections
0,Bayberry Commons Nursing & Rehabilitation Center,1
1,Landmark Medical Center,1
2,Rhode Island Hospital,3
3,Roger Williams Medical Center,3
4,Scandinavian Home,1
5,The Miriam Hospital,1
6,Westerly Hospital,1


### Adult Day Center

In [20]:
def adc_census_on_date(params):
    return u.dataframe_query("""SELECT * FROM authorizations
                WHERE service_type='Adult Day Center Attendance'
                AND (approval_expiration_date >= ?
                OR approval_expiration_date IS NULL)
                AND approval_effective_date <= ?""",
                              params).groupby("vendor").count()["member_id"].reset_index().rename(columns={"member_id":
                                                                                                       f"adc_census-{params[0][:7]}"})

In [21]:
alf_census_on_date(("2019-01-01", "2019-01-30"))

,facility_name,census-2019-01
0,Arbor Hill Assisted Living Community,5
1,Autumn Villa,1
2,Darlington Memory Lane,2
3,Highlands on the East Side,7
4,Spring Villa Dementia Care,5
5,St. Elizabeth Court,7
6,St. Germain Assisted Living,1
7,The Villa at Saint Antoine,1


### Pressure Ulcers

In [22]:
def pressure_ulcers_at_facility(facility_type, params):
    params = [facility_type] + list(params)
    df = u.dataframe_query("""SELECT * FROM wounds
                        WHERE living_situation IS ?
                        AND date_time_occurred BETWEEN ? AND ?""",
                      params).groupby("living_detail").count()["member_id"].reset_index().rename(columns={"member_id": f"{facility_type}_pulcers-{params[0][:7]}"})
    df["living_detail"] = df["living_detail"].fillna("Unknown")
    return df

In [23]:
pressure_ulcers_at_facility("SNF", ("2018-01-01", "2019-10-30"))

,living_detail,SNF_pulcers-SNF
0,Bayberry,3
1,Berkshire,13
2,Crestwood,5
3,Riverview,1
4,Scandinavian,2
5,St Antoine,1
6,St. Antoine's,7
7,Steere,13
8,West Shore,3


### Building the CSVs

In [24]:
build_csv_date_spread(alf_census_on_date, "alfs", "facility_name", "2018-01-01", "2019-10-31").to_csv("output/alf_census.csv",
                                                                                                   index=False)
build_csv_date_spread(alf_to_hosp, "alfs", "facility_name", "2018-01-01", "2019-10-31").to_csv("output/hosp_from_alf.csv",
                                                                                               index=False)

build_csv_date_spread(nf_census_on_date, "nursing_home", "facility", "2018-01-01", "2019-10-31").to_csv("output/nf_census.csv",
                                                                                                     index=False)
build_csv_date_spread(nf_to_hosp, "nursing_home", "facility", "2018-01-01", "2019-10-31").to_csv("output/hosp_from_nf.csv",
                                                                                                 index=False)

build_csv_date_spread(hosp_admissions, "inpatient", "facility", "2018-01-01", "2019-10-31").to_csv("output/hosp_admissions.csv",
                                                                                               index=False)
build_csv_date_spread(resulting_30_day_hosp_count, "inpatient", "facility", "2018-01-01", "2019-10-31").to_csv("output/hosp_admit_results_in_30day.csv",
                                                                                                           index=False)
build_csv_date_spread(readmit_30_day_hosp_count, "inpatient", "facility", "2018-01-01", "2019-10-31").to_csv("output/hosp_30_day_readmits.csv",
                                                                                                         index=False)
build_csv_date_spread(infections_by_hosp, "inpatient", "facility", "2018-01-01", "2019-10-31").to_csv("output/hosp_infections.csv",
                                                                                                  index=False)

build_csv_date_spread(adc_census_on_date, "authorizations", "vendor", "2018-01-01", "2019-10-31").to_csv("output/adc_census.csv",
                                                                                                      index=False)

build_pressure_wound_csv("SNF", "2018-01-01", "2019-11-20").to_csv("output/snf_pulcers.csv", index=False)
build_pressure_wound_csv("ALF", "2018-01-01", "2019-11-20").to_csv("output/alf_pulcers.csv", index=False)